In [ ]:
# Steel Industry Energy Consumption Prediction ML Project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, PowerTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, mutual_info_regression, RFE
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from collections import Counter
from sklearn.utils import resample
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
print("Loading Steel Industry Energy Consumption Dataset...")
df = pd.read_csv('Steel_industry_data.csv')
print(f"Dataset shape: {df.shape}")
print("\nDataset Info:")
print(df.info())
print("\nFirst few rows:")
print(df.head())

# 1. CHECK MISSING VALUES AND TREAT THEM
print("\n" + "="*50)
print("STEP 1: MISSING VALUES ANALYSIS")
print("="*50)

print("\nMissing values count:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

if missing_values.sum() > 0:
    # Impute numerical features with median
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    imputer_num = SimpleImputer(strategy='median')
    df[numeric_cols] = imputer_num.fit_transform(df[numeric_cols])

    # Impute categorical features with most frequent
    cat_cols = df.select_dtypes(include=['object']).columns
    if len(cat_cols) > 0:
        imputer_cat = SimpleImputer(strategy='most_frequent')
        df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

    print("Missing values treated successfully!")
else:
    print("No missing values found!")

# 2. CHECK FOR DATA IMBALANCE (For target variable if categorical)
print("\n" + "="*50)
print("STEP 2: DATA DISTRIBUTION ANALYSIS")
print("="*50)

# Analyze target variable (Usage_kWh - this appears to be the target for regression)
target_col = 'Usage_kWh'
print(f"\nTarget variable ({target_col}) statistics:")
print(df[target_col].describe())

# Check for any categorical columns that might need balancing
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\n{col} distribution:")
    print(df[col].value_counts())

# If Load_Type has severe imbalance, we can address it
if 'Load_Type' in df.columns:
    load_type_counts = df['Load_Type'].value_counts()
    print(f"\nLoad_Type distribution:\n{load_type_counts}")

    # Check if balancing is needed (if imbalance ratio > 3:1)
    max_count = load_type_counts.max()
    min_count = load_type_counts.min()
    if max_count / min_count > 3:
        print("Significant imbalance detected in Load_Type. Applying SMOTE-like balancing...")
        # For regression, we'll use stratified sampling based on Load_Type
        # This is more appropriate than upsampling for regression tasks

# 3. FEATURE SCALING (Applied later after feature engineering)
print("\n" + "="*50)
print("STEP 3: FEATURE SCALING (Will be applied after feature engineering)")
print("="*50)

# 4. OUTLIER DETECTION AND TREATMENT
print("\n" + "="*50)
print("STEP 4: OUTLIER DETECTION AND TREATMENT")
print("="*50)

numeric_cols = df.select_dtypes(include=[np.number]).columns
outlier_info = {}

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    outlier_info[col] = outliers_count

    if outliers_count > 0:
        print(f"{col}: {outliers_count} outliers detected")
        # Cap outliers instead of removing them
        df[col] = np.clip(df[col], lower_bound, upper_bound)

print(f"Outliers treated using IQR method with capping")

# 5. ENCODE CATEGORICAL FEATURES
print("\n" + "="*50)
print("STEP 5: CATEGORICAL FEATURE ENCODING")
print("="*50)

# Remove date column if present (not useful for modeling)
if 'date' in df.columns:
    df = df.drop('date', axis=1)
    print("Date column removed")

categorical_cols = df.select_dtypes(include=['object']).columns
print(f"Categorical columns found: {list(categorical_cols)}")

# Apply appropriate encoding
if len(categorical_cols) > 0:
    # For ordinal features like Load_Type, use Label Encoding
    # For nominal features, use One-Hot Encoding

    ordinal_cols = ['Load_Type']  # Assuming Load_Type might have some order
    nominal_cols = [col for col in categorical_cols if col not in ordinal_cols]

    # Label Encoding for ordinal
    le = LabelEncoder()
    for col in ordinal_cols:
        if col in df.columns:
            df[col] = le.fit_transform(df[col])
            print(f"Label encoded: {col}")

    # One-Hot Encoding for nominal
    for col in nominal_cols:
        if col in df.columns:
            df = pd.get_dummies(df, columns=[col], drop_first=True)
            print(f"One-hot encoded: {col}")

print("Reason: Label encoding for ordinal features preserves order relationships,")
print("while one-hot encoding for nominal features avoids imposing false ordinality.")

# 6. FEATURE ENGINEERING
print("\n" + "="*50)
print("STEP 6: FEATURE ENGINEERING")
print("="*50)

# Create time-based features if date information is available in other columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.drop(target_col)

# Create lag features for time series patterns
print("Creating lag features...")
for col in numeric_cols[:5]:  # Limit to avoid too many features
    df[f'{col}_lag1'] = df[col].shift(1).fillna(df[col].mean())

# Create rolling window features
print("Creating rolling window features...")
for col in numeric_cols[:5]:
    df[f'{col}_rolling_mean3'] = df[col].rolling(window=3, min_periods=1).mean()
    df[f'{col}_rolling_std3'] = df[col].rolling(window=3, min_periods=1).std().fillna(0)

# Create interaction features between important variables
print("Creating interaction features...")
if 'Lagging_Current_Reactive.Power_kVarh' in df.columns and 'Leading_Current_Reactive_Power_kVarh' in df.columns:
    df['Total_Reactive_Power'] = df['Lagging_Current_Reactive.Power_kVarh'] + df['Leading_Current_Reactive_Power_kVarh']

# Create ratio features
if 'Lagging_Current_Power_Factor' in df.columns and 'Leading_Current_Power_Factor' in df.columns:
    df['Power_Factor_Ratio'] = df['Lagging_Current_Power_Factor'] / (df['Leading_Current_Power_Factor'] + 1e-8)

print(f"Feature engineering completed. New shape: {df.shape}")

# 7. FEATURE SELECTION
print("\n" + "="*50)
print("STEP 7: FEATURE SELECTION")
print("="*50)

X = df.drop(target_col, axis=1)
y = df[target_col]

print("Method 1: Mutual Information for feature selection")
print("Reason: Mutual information captures both linear and non-linear relationships")
print("between features and target, making it suitable for regression tasks.")

# Apply mutual information
selector_mi = SelectKBest(mutual_info_regression, k=15)  # Select top 15 features
X_selected_mi = selector_mi.fit_transform(X, y)
selected_features_mi = X.columns[selector_mi.get_support()]

print(f"Selected features using Mutual Information: {len(selected_features_mi)}")
print(list(selected_features_mi))

# Method 2: Recursive Feature Elimination with Random Forest
print("\nMethod 2: RFE with Random Forest")
print("Reason: RFE with tree-based models can identify feature importance")
print("while considering feature interactions.")

rf_selector = RandomForestRegressor(n_estimators=100, random_state=42)
rfe_selector = RFE(rf_selector, n_features_to_select=15)
rfe_selector.fit(X, y)
selected_features_rfe = X.columns[rfe_selector.support_]

print(f"Selected features using RFE: {len(selected_features_rfe)}")
print(list(selected_features_rfe))

# Combine both selections (union of features)
final_features = list(set(selected_features_mi) | set(selected_features_rfe))
X_final = X[final_features]

print(f"\nFinal features after combining both methods: {len(final_features)}")

# 8. FEATURE TRANSFORMATION
print("\n" + "="*50)
print("STEP 8: FEATURE TRANSFORMATION")
print("="*50)

# Apply scaling
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_final),
                       columns=X_final.columns,
                       index=X_final.index)

# Apply power transformation for normalization
pt = PowerTransformer(method='yeo-johnson')
X_transformed = pd.DataFrame(pt.fit_transform(X_scaled),
                           columns=X_scaled.columns,
                           index=X_scaled.index)

print("Applied StandardScaler and PowerTransformer (Yeo-Johnson)")
print("Reason: StandardScaler ensures all features have equal importance,")
print("while PowerTransformer helps achieve normality in feature distributions.")

# 9. TRAIN-TEST SPLIT
print("\n" + "="*50)
print("STEP 9: TRAIN-TEST SPLIT")
print("="*50)

X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y, test_size=0.2, random_state=42
)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# 10. MODEL SELECTION WITH CROSS VALIDATION
print("\n" + "="*50)
print("STEP 10: MODEL SELECTION WITH CROSS VALIDATION")
print("="*50)

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(random_state=42),
    'Lasso Regression': Lasso(random_state=42),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'KNN': KNeighborsRegressor(),
    'SVR': SVR()
}

cv_results = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) if len(np.unique(y)) < 10 else 5

print("Cross-validation results (RMSE):")
print("-" * 40)

for name, model in models.items():
    try:
        scores = cross_val_score(model, X_train, y_train,
                               cv=5, scoring='neg_root_mean_squared_error')
        cv_results[name] = -scores.mean()
        print(f"{name:20}: {cv_results[name]:.4f} (+/- {scores.std() * 2:.4f})")
    except Exception as e:
        print(f"{name:20}: Error - {str(e)}")

# Find best model
best_model_name = min(cv_results, key=cv_results.get)
best_model = models[best_model_name]

print(f"\nBest model: {best_model_name} (RMSE: {cv_results[best_model_name]:.4f})")

# 11. FINAL TRAINING
print("\n" + "="*50)
print("STEP 11: FINAL TRAINING OF BEST MODEL")
print("="*50)

best_model.fit(X_train, y_train)
print(f"Best model ({best_model_name}) trained on full training set")

# 12. EVALUATION
print("\n" + "="*50)
print("STEP 12: COMPREHENSIVE EVALUATION")
print("="*50)

# Predictions
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate metrics
def calculate_metrics(y_true, y_pred, set_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"\n{set_name} Set Metrics:")
    print(f"MSE:  {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE:  {mae:.4f}")
    print(f"R²:   {r2:.4f}")

    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2}

train_metrics = calculate_metrics(y_train, y_train_pred, "Training")
test_metrics = calculate_metrics(y_test, y_test_pred, "Test")

# Check for overfitting
print(f"\nOverfitting Analysis:")
print(f"Training R²: {train_metrics['R2']:.4f}")
print(f"Test R²:     {test_metrics['R2']:.4f}")
print(f"Difference:  {train_metrics['R2'] - test_metrics['R2']:.4f}")

if train_metrics['R2'] - test_metrics['R2'] > 0.1:
    print("⚠️  Model may be overfitting")
else:
    print("✅ Model shows good generalization")

# 13. HYPERPARAMETER TUNING
print("\n" + "="*50)
print("STEP 13: HYPERPARAMETER TUNING")
print("="*50)

# Define parameter grids for top models
param_grids = {
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
    }
}

if best_model_name in param_grids:
    print(f"Tuning hyperparameters for {best_model_name}...")

    param_grid = param_grids[best_model_name]

    # Use smaller parameter grid for faster execution
    if best_model_name == 'Random Forest':
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [None, 20],
            'min_samples_split': [2, 5]
        }

    grid_search = GridSearchCV(
        best_model, param_grid, cv=5,
        scoring='neg_root_mean_squared_error', n_jobs=-1
    )

    grid_search.fit(X_train, y_train)

    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV score: {-grid_search.best_score_:.4f}")

    # Update best model
    best_model_tuned = grid_search.best_estimator_

else:
    print(f"No hyperparameter tuning defined for {best_model_name}")
    best_model_tuned = best_model

# 14. FINAL MODEL VALIDATION
print("\n" + "="*50)
print("STEP 14: FINAL MODEL VALIDATION")
print("="*50)

# Final predictions with tuned model
y_train_pred_final = best_model_tuned.predict(X_train)
y_test_pred_final = best_model_tuned.predict(X_test)

print("Final Model Performance:")
final_train_metrics = calculate_metrics(y_train, y_train_pred_final, "Training")
final_test_metrics = calculate_metrics(y_test, y_test_pred_final, "Test")

# Feature importance (if available)
if hasattr(best_model_tuned, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': best_model_tuned.feature_importances_
    }).sort_values('importance', ascending=False)

    print(f"\nTop 10 Most Important Features:")
    print(feature_importance.head(10))

# Final model summary
print("\n" + "="*60)
print("FINAL MODEL SUMMARY")
print("="*60)
print(f"Best Model: {best_model_name}")
print(f"Test RMSE: {final_test_metrics['RMSE']:.4f}")
print(f"Test R²: {final_test_metrics['R2']:.4f}")
print(f"Number of features: {X_train.shape[1]}")

# Save the model and preprocessors (if needed)
import joblib

# Save the complete pipeline components
model_artifacts = {
    'model': best_model_tuned,
    'scaler': scaler,
    'power_transformer': pt,
    'selected_features': final_features,
    'feature_selector_mi': selector_mi,
    'rfe_selector': rfe_selector
}

# Uncomment to save
# joblib.dump(model_artifacts, 'steel_energy_model_artifacts.pkl')
# print("\nModel artifacts saved to 'steel_energy_model_artifacts.pkl'")

print("\n✅ Steel Industry Energy Consumption Prediction Model Complete!")
print("="*60)

Loading Steel Industry Energy Consumption Dataset...
Dataset shape: (35040, 11)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   date                                  35040 non-null  object 
 1   Usage_kWh                             35040 non-null  float64
 2   Lagging_Current_Reactive.Power_kVarh  35040 non-null  float64
 3   Leading_Current_Reactive_Power_kVarh  35040 non-null  float64
 4   CO2(tCO2)                             35040 non-null  float64
 5   Lagging_Current_Power_Factor          35040 non-null  float64
 6   Leading_Current_Power_Factor          35040 non-null  float64
 7   NSM                                   35040 non-null  int64  
 8   WeekStatus                            35040 non-null  object 
 9   Day_of_week                           35040 non-null  